In [1]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import timedelta
import datetime
import urllib.parse
import numpy as np
import pandas as pd
from time import sleep
import pickle

import os
import glob

In [2]:
os.getcwd()

'C:\\Users\\TAK\\Desktop\\Data\\Lectures\\Others\\coding\\기타 코딩\\1905 퀴퍼 셋리스트 크롤링'

In [3]:
glob.glob("*.*")

['190503 데이터크롤링_alpha.ipynb',
 '190503 데이터크롤링_beta.ipynb',
 '190503 퀴퍼 셋리스트 크롤링 v.1.ipynb',
 'input_test.xlsx',
 '셋리스트(to.나무거북이).xlsx']

In [4]:
#잘 불러와지나 테스트

df0=pd.read_excel('input_test.xlsx',header = 0,encoding='UTF-8').fillna(0) #header : 앞에 1줄 제외하려고. fillna(0)은 빈칸을 0으로 만들기 위해서
#resd_excel에서 csv파일을 위와 같은 형태의 식으로 부르면 오류가 생김. csv 파일에서는 header 기능을 쓸 수 없을 것으로 보이는 데 그 때문 같다.

df0.head()

,Num,Input_artist,Input_Name
0,1,김수희,남행열차
1,2,소녀시대,다시 만난 세계
2,3,Queen,We will rock you
3,4,코리아나,손에 손 잡고
4,5,이선희,아름다운 강산


In [5]:
n=len(df0.index)

In [6]:
from selenium import webdriver #리모컨 가져온다는 느낌

In [7]:
output=[]
browser=webdriver.Chrome('C:/chromedriver/chromedriver.exe')


for i in range(n) :
    
    output_temp=[]

    inArtist = df0.iloc[i][1]
    inSongName = df0.iloc[i][2]
    
    search_key = inArtist+" "+inSongName
    
#     search_key_code=""
#     for _ in range(len(search_key)) :
        
#         if search_key[_]==" " :
#             search_key_code=search_key_code+"%20"
#         else :
#             temp=str(search_key[_].encode("utf-8"))
#             temp_split=temp.split("\\x")
#             for j in range(len(temp_split)) :
#                 if j == len(temp_split)-1 :
#                     search_key_code=search_key_code+"%"+temp_split[j][:-1]
#                 elif j != 0 :
#                     search_key_code=search_key_code+"%"+temp_split[j]
        
#     url="https://www.genie.co.kr/search/searchMain?query="+search_key_code.upper()

# 위 코드가 문제 생긴 이유 : 영문자는 utf-8 인코딩이 안한채로 주소가 적용되어서.
    
    # 제어용 웹 드라이버 설정

    url="https://www.genie.co.kr/search/searchMain?query="+search_key
    browser.get(url)
    url=browser.current_url #크롬 드라이버에서 현재 접속해 있는 주소
        
        
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
        
    if len(soup.find_all('tr', attrs={'class':'list'}))==0 or soup.find_all('tr', attrs={'class':'list'})[0].text=="\n등록된 곡이 없습니다.\n":
        output_temp=["N/A","N/A","N/A"]
    
    else :
        AA=str(soup.find_all('tr', attrs={'class':'list'})[0]) #현재 노래 별로 block이 잡혀있어서, 이렇게 하면 제일 먼저 검색된 게 올라옴
        
        #노래 ID 추출하기 - 지니의 경우 노래 id별로 url이 되어있어서 
        BB=AA.split('<')
        CC=BB[1]
        k = CC.find('songid')+8
        SongID=CC[k:CC.find('"',k)] #근데 이렇게 안하고 그냥 숫자만 추출하기로 해도 되려나 싶음.
        
        url = "https://www.genie.co.kr/detail/songInfo?xgnm="+SongID
        
        #songid로 지정된 페이지에서 파싱
        resp = requests.get(url, headers = headers)
        soup1 = BeautifulSoup(resp.text, 'html.parser')
        
        #곡 이름 추출
        FF=str(soup1.find_all('h2', attrs={'class':'name'})[0].text)
        kkk=FF.find("\t")
        
        a=[]
        index=kkk
        done = False
        while done == False :
            if index >= len(FF) :
                done = True
            elif FF[index:index+1]=="\t" :
                index+=2
            else :
                a.append(index)
                index+=1
        
        SongName_result=""
        for _ in range(len(a)) :
            SongName_result=SongName_result+FF[a[_]-1]
            
        #곡 관련 세부 정보 : 가수, 앨범, 장르, 재생시간, +a 순\
        DD=str(soup1.find_all('span', attrs={'class':'value'})[0])
        EE=str(soup1.find_all('span', attrs={'class':'value'})[3])
        
        t=EE.find(":")
        Songtime_result=EE[t-2:t+3]
        
        GG=DD.split(">")[-3]
        aa=DD.split(">")[-3].find("<")
        Artistname_result=DD.split(">")[-3][:aa]
        
        output_temp=[Artistname_result,SongName_result,Songtime_result]
        
    output.append(output_temp)
    
print(output)  
    

[['김수희', '남행열차', '02:57'], ["소녀시대 (GIRLS' GENERATION)", '다시 만난 세계 (Into The New World)', '04:25'], ['The Royal Philharmonic Pop Orchestra', "QUEEN : We Will Rock You (뮤지컬 '맘마미아')", '03:14'], ['코리아나 (Koreana)', '손에 손잡고', '04:13'], ['이선희', '아름다운 강산', '05:58'], ["소녀시대 (GIRLS' GENERATION)", "소녀시대 (Girls' Generation)", '03:50'], ['김건모', '잘못된 만남', '04:18'], ['소찬휘', 'Tears', '03:57'], ['김현정', '멍', '03:28'], ['이정현', '와', '03:33'], ['모두의MR', '달빛의 전설 (달의 요정 세일러문 OST) (김현아) (MR)', '02:49'], ['자우림', '매직 카펫 라이드', '03:32'], ['보아 (BoA)', 'No. 1', '03:11'], ['체리 필터 (Cherry Filter)', '낭만 고양이', '03:48'], ['보아 (BoA)', '공중정원(Garden In The Air)', '03:51'], ['원더걸스', "Tell Me (Sampling From 'Two Of Hearts')", '03:37'], ['원더걸스', 'Nobody', '03:33'], ['이효리', 'U-Go-Girl (With. 낯선)', '03:08'], ['이효리', 'Chitty Chitty Bang Bang (Feat. Ceejay Of Freshboyz)', '03:24'], ["소녀시대 (GIRLS' GENERATION)", 'Gee', '03:22'], ["소녀시대 (GIRLS' GENERATION)", '소원을 말해봐 (Genie)', '03:49'], ['2NE1', '내가 제일 잘 나가', '03:29'], ['2NE1', 'Fir

In [10]:
df=df0.copy()

Artist_Result =[]
Song_Result = []
Time_Result =[]

for i in range(len(output)) :
    Artist_Result.append(output[i][0])
    Song_Result.append(output[i][1])
    Time_Result.append(output[i][2])
    
df["Artist_Result"]=Artist_Result
df["Song_Result"]=Song_Result
df["Time_Result"]=Time_Result


In [11]:
display(df)

,Num,Input_artist,Input_Name,Artist_Result,Song_Result,Time_Result
0,1,김수희,남행열차,김수희,남행열차,02:57
1,2,소녀시대,다시 만난 세계,소녀시대 (GIRLS' GENERATION),다시 만난 세계 (Into The New World),04:25
2,3,Queen,We will rock you,The Royal Philharmonic Pop Orchestra,QUEEN : We Will Rock You (뮤지컬 '맘마미아'),03:14
3,4,코리아나,손에 손 잡고,코리아나 (Koreana),손에 손잡고,04:13
4,5,이선희,아름다운 강산,이선희,아름다운 강산,05:58
5,6,소녀시대,소녀시대,소녀시대 (GIRLS' GENERATION),소녀시대 (Girls' Generation),03:50
6,7,김건모,잘못된 만남,김건모,잘못된 만남,04:18
7,8,소찬휘,Tears,소찬휘,Tears,03:57
8,9,김현정,멍,김현정,멍,03:28
9,10,이정현,와,이정현,와,03:33


In [12]:
df.to_excel('search result.xlsx',encoding='utf-8')